In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
loans = pd.read_csv('kiva_loans.csv')
#loans.head()
loans.columns


In [ ]:
lenders = pd.read_csv('lenders.csv')


In [ ]:
loans = lns.drop(['repayment_interval','date','use','id','tags', 
                  'funded_amount','term_in_months','partner_id',
                  'region','posted_time', 'disbursed_time', 'funded_time',],axis=1)
loans = loans.fillna('unknown')
loans.loc[loans['borrower_genders'].str.contains('emale'), 'borrower_genders'] = 'female'
loans.loc[loans['borrower_genders'].str.contains(', male'), 'borrower_genders'] = 'male'

loans.borrower_genders.value_counts()

In [ ]:
loans.head()

In [ ]:
lenders.columns

In [ ]:
lenders = lenders.drop(['permanent_name', 'display_name', 'city', 'state',
                 'member_since', 'occupation', 'loan_because',
                 'invited_by', 'num_invited'],axis=1)
lenders.rename(columns= {"country_code":"state"}, inplace=True)
lenders = lenders[pd.isnull(lenders['state']) == False]
lenders = lenders.groupby('state').filter(lambda x : len(x)>300)

In [ ]:
print(lenders.shape)
print(lenders.state.value_counts())

In [ ]:
themes = pd.read_csv('loan_themes_by_region.csv')

In [ ]:
print(themes.shape)
print(themes.columns)

In [ ]:
themes = themes.drop(['LocationName', 'geocode', 'names', 'geo', 'lat',
       'lon', 'mpi_region', 'mpi_geo', 'rural_pct','forkiva', 'region', 
                      'geocode_old', 'ISO','number','Partner ID', 
                      'Field Partner Name', 'sector', 'Loan Theme ID' ],axis=1)
themes.rename(columns= {'Loan Theme Type':"theme"}, inplace=True)
themes = themes.groupby('country').filter(lambda x : len(x)>130)

In [ ]:
amounts = themes.groupby('country').agg(sum)
amounts = amounts.sort_values(by=['amount'],ascending=False)
amounts

In [ ]:
themes.loc[themes['theme'].str.contains('Mothers'), 'theme'] = 'Women'
themes.loc[themes['theme'].str.contains('Clean'), 'theme'] = 'Green'
themes.loc[themes['theme'].str.contains('Vulnerable'), 'theme'] = 'Underserved'
themes.loc[themes['theme'].str.contains('Chance'), 'theme'] = 'Underserved'
themes.loc[themes['theme'].str.contains('Finance'), 'theme'] = 'Finance'
themes.loc[themes['theme'].str.contains('bank'), 'theme'] = 'Finance'
themes.loc[themes['theme'].str.contains('Loan'), 'theme'] = 'Finance'
themes.loc[themes['theme'].str.contains('Youth'), 'theme'] = 'Youth'
themes.loc[themes['theme'].str.contains('Transactions'), 'theme'] = 'Finance'
themes.loc[themes['theme'].str.contains('Agriculture'), 'theme'] = 'Agriculture'
themes.loc[themes['theme'].str.contains('Mothers'), 'theme'] = 'Women'
themes.loc[themes['theme'].str.contains('Water'), 'theme'] = 'Water'
themes.loc[themes['theme'].str.contains('ducation'),'theme'] = 'Education'
themes.loc[themes['theme'].str.contains('Water'), 'theme'] = 'Water'
themes.loc[themes['theme'].str.contains('mothers'), 'theme'] = 'Women'
themes.loc[themes['theme'].str.contains('Underserved'), 'theme'] = 'Underserved'
themes.loc[themes['theme'].str.contains('Artisan'), 'theme'] = 'SME'
themes.loc[themes['theme'].str.contains('Bio'), 'theme'] = 'Green'
themes.loc[themes['theme'].str.contains('loan'), 'theme'] = 'Finance'
themes.loc[themes['theme'].str.contains('Sanitation'), 'theme'] = 'Water'
themes.loc[themes['theme'].str.contains('credit'), 'theme'] = 'Finance'
themes.loc[themes['theme'].str.contains('Solar'), 'theme'] = 'Green'
themes.loc[themes['theme'].str.contains('Small'), 'theme'] = 'SME'
themes.loc[themes['theme'].str.contains('gricultural'), 'theme'] = 'Agriculture'
themes.loc[themes['theme'].str.contains('Displaced'), 'theme'] = 'Housing'
themes.loc[themes['theme'].str.contains('Housing'), 'theme'] = 'Housing'
themes.loc[themes['theme'].str.contains('omen'), 'theme'] = 'Women'

#for i, row in themes[['theme']].iterrows():
 #   if ('Women' in themes.at[i,'theme']) == True : 
  #    themes.at[i,'theme'] = 'Women'

In [ ]:
themes = themes.groupby('theme').filter(lambda x : len(x)>100)


In [ ]:
import requests
from bs4 import BeautifulSoup


def urls(url):
  r = requests.get(url)
  r.content
  soup= BeautifulSoup(r.content, 'html.parser')
  return soup

In [ ]:

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita'
  
soup = urls(url)
  
table=soup.find_all('table',{'class':'wikitable'})

rows=table[0].find_all('tr')
rows=[row.text.strip().split('\n') for row in rows]


In [ ]:
def restr(x):
    if len(x)>4:
        x[3]=='\n'.join(x[3:])
        del x[4:]
    elif len(x)==3:
        x.append("")
    return x
rows=[restr(x) for x in rows]


In [ ]:
df=pd.DataFrame([len(x) for x in rows])
df[0].value_counts()
colnames = rows[0]
data = rows[1:]

df = pd.DataFrame(data, columns=colnames)
df.rename(index=str, columns={'':'Country',
                                'Country/Territory':'GDP'}, inplace=True)

df1 = df.iloc[:,0:3]

In [ ]:
df1.Country=[' '.join(i) for i in df1.Country.str.split()]
#\xa0 white space erased
# #df2['Country'] =[ i.replace('\xa0', '') for i in df2['Country']]


In [ ]:
merged = df1.merge(amounts, left_on='Country', right_on='country', how='inner')
merged = merged.drop(['country'],axis=1)
merged = merged.sort_values('amount')

In [ ]:
url2 = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
  
soup2 = urls(url2)
  
table2=soup2.find_all('table',{'class':'wikitable'})
rows2=table2[0].find_all('tr')
rows2=[row.text.strip().split('\n') for row in rows2]

def essai(x):
    if len(x)>4:
        x[3]=='\n'.join(x[5:])
        del x[5:]
    elif len(x)==3:
        x.append("")
    return x

rows2=[essai(x) for x in rows2]

p=pd.DataFrame([len(x) for x in rows2])
p[0].value_counts()
colnames = rows2[0]
data = rows2[1:]
peeps = pd.DataFrame(data, columns=colnames)

In [ ]:
pops = peeps.drop(['Rank', 'Date','Country(or dependent territory)'
       ],axis=1)
pops.rename(index=str, columns={'% of worldpopulation':'Population',
                               'Population':'Country' 
                             }, inplace=True)

In [ ]:
import re
pops.Country = pops.Country.apply(lambda x : (re.split(r'\[',x)[0].strip()))

In [ ]:
df3 = merged.merge(pops, left_on='Country', right_on='Country', how='inner')


In [ ]:
df3.Population=df3.Population.str.replace(",","").astype(int)
df3.GDP=df3.GDP.str.replace(",","").astype(int)
df3.Rank=df3.Rank.astype(int)

In [ ]:
df3['Funding_p1000'] = df3.amount / df3.Population * 1000


In [ ]:
countries = pd.DataFrame(themes.country.value_counts().reset_index())
countries.columns = ['countries', 'count']

df4 = df3.merge(countries, left_on='Country', right_on='countries', how='inner')

In [ ]:
df4 = df4.drop(['countries'],axis=1)
df4['Avg_funding'] = round(df4.amount/df4['count'],0)
df4['Projects_p1MLN'] = round(df4['count']*1000000/df4.Population,1)


In [ ]:
df4.boxplot(column='Projects_p1MLN')

In [ ]:

df4 = df4.loc[df4['Projects_p1MLN']!=df4['Projects_p1MLN'].max()]
df4.boxplot(column='Funding_p1000')

In [ ]:

df4 = df4.loc[df4['Funding_p1000']!=df4['Funding_p1000'].max()]

df4.boxplot(column='amount')


In [ ]:
df4 = df4.loc[df4['amount']!=df4['amount'].max()]
df4.boxplot(column='Avg_funding')


In [ ]:
df8 = df4.loc[df4['Avg_funding']!=df4['Avg_funding'].max()]


# number of projects per country for 1 million people

df8.head()

In [ ]:
sns.heatmap(df8.corr(), cmap='YlGnBu')


In [ ]:
df8.corr()

In [ ]:
sns.lmplot('Avg_funding','amount',data=df8)
plt.xlim(0, 65000)

plt.ylim(0, 21000000)

In [ ]:
sns.lmplot('count','amount',data=df8)
plt.xlim(0, 900)
plt.ylim(0, 21000000)

In [ ]:
sns.jointplot(y='amount',x='Funding_p1000',data=df8, kind='reg')



http://www.uta.fi/yky/en/studies/disciplines/northamericanstudies/summer_school/2013_Yan_Chen_Kiva_Experiment_2013_06_60min.pdf https://hackernoon.com/community-focused-growth-8a4c5a686def 
slides https://slideplayer.com/slide/4061274/

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
X=df8.assign(const=1)
X = X.drop(['Country'], axis=1)
pd.Series([vif(X.values,i) for i in range(X.shape[1])],index=X.columns)

In [ ]:
import statsmodels.formula.api as smf

def get_vif(exogs,data):
    vif_array=np.array([])
    tolerance_array=np.array([])
    
    for exog in exogs:
        not_exog=[i for i in exogs if i!=exog]
        formula=f"{exog} ~ {' + '.join(not_exog)}"
        
        r_squared=smf.ols(formula,data=data).fit().rsquared
        
        vif=1/(1-r_squared)
        vif_array=np.append(vif_array,vif).round(2)
        
        tolerance=1-r_squared
        tolerance_array=np.append(tolerance_array,tolerance).round(2)
    df_vif=pd.DataFrame({'VIF':vif_array,'Tolerance':tolerance_array},index=exogs)
    
    return df_vif

In [ ]:
import numpy as np
df9 = df8.drop(['Country'], axis=1)
df9 = df9.drop(['Rank'], axis=1)
get_vif(exogs=df9.columns,data=df8)

In [ ]:
df9=(df9-df9.mean())/df9.std()
df9.shape

In [ ]:

results = smf.ols(' Avg_funding ~ Projects_p1MLN+amount', data=df9).fit()

# Inspect the results
print(results.summary())

In [ ]:
loans.head()

In [ ]:
loans2 = loans.loc[loans['borrower_genders']!='unknown']
loans2.head()


In [ ]:
amounts_pG = loans2.groupby(['country','borrower_genders']).agg(sum)
amounts_pG = amounts_pG.reset_index()

amounts_pG=amounts_pG.drop(['activity', 'sector', 'country_code', 'currency'],axis=1)
amounts_pG.columns

In [ ]:
amounts_pG['isFemale']=[1 if i == 'female' else 0 for i in amounts_pG.borrower_genders]
amounts_pG['Avg_funding'] = round(amounts_pG.loan_amount/amounts_pG['lender_count'],0)
amounts_pG = amounts_pG.sort_values('lender_count' )
amounts_pG = amounts_pG.loc[amounts_pG['lender_count']>1000]

amounts_pG.head()


In [ ]:
amounts_pG = amounts_pG.drop(['country'],axis=1)
amounts_pG = amounts_pG.drop(['borrower_genders'],axis=1)


In [ ]:
amounts_pG.boxplot(column='lender_count')

In [ ]:
amounts_pG = amounts_pG.loc[amounts_pG['lender_count']!=0]
amounts_pG = amounts_pG.loc[amounts_pG['Avg_funding']<50]
amounts_pG = amounts_pG.loc[amounts_pG['loan_amount']<9000000]
amounts_pG = amounts_pG.loc[amounts_pG['lender_count']<500000]